In [163]:
# In order to run this script, ensure you have installed the required dependencies in the requirements.txt file.
# For more information, refer to the README.md file.
import pandas as pd
import numpy as np
import os
from scipy.interpolate import interp1d

We label appropriately the columns we need from the CSVs.

We also load the `.csv` files data set from the `/data/` folder of the repository.

Finally we prepare the data set to use timestamp as index of the dataframe.

In [164]:
WATER_LEVEL_COL="Water Level (meters) - Niveau d'eau (metres)"
DATE_COL="Date - Date"
OUTFLOW_COL="m^3/s"

# load csvs
cornwall_water_lvl = pd.read_csv("./data/raw/Cornwall_WaterLevelChanges_2024_CLEAN.csv")
longsault_water_lvl = pd.read_csv("./data/raw/LongSaultDam_WaterLevelChanges_2024_CLEAN.csv")
lakeontario_outflow_changes = pd.read_csv("./data/raw/LakeOntarioOutflowChanges_2024_CLEAN.csv")

# convert date columns to datetime
cornwall_water_lvl[DATE_COL] = pd.to_datetime(cornwall_water_lvl[DATE_COL])
longsault_water_lvl[DATE_COL] = pd.to_datetime(longsault_water_lvl[DATE_COL])
lakeontario_outflow_changes[DATE_COL] = pd.to_datetime(lakeontario_outflow_changes[DATE_COL])

# set date columns as index
cornwall_water_lvl.set_index(DATE_COL, inplace=True)
longsault_water_lvl.set_index(DATE_COL, inplace=True)
lakeontario_outflow_changes.set_index(DATE_COL, inplace=True)

Add datums for normal elevation on longsault and cornwall

In [165]:
# adjust water levels by adding normal elevation levels
longsault_water_lvl[WATER_LEVEL_COL] = longsault_water_lvl[WATER_LEVEL_COL] + 72.5 
cornwall_water_lvl[WATER_LEVEL_COL] = cornwall_water_lvl[WATER_LEVEL_COL] + 46.4

Since the data recorded from lake 

In [166]:
# reindex to hourly frequency and fill missing timestamps with the previous known value
start = lakeontario_outflow_changes.index[0]
# end is set to the last timestamp in 2024 plus one minute since datasets are recorded hourly plus one minute
end = pd.Timestamp("2024-12-31 23:01:00")

hourly_index = pd.date_range(start=start, end=end, freq="h")
lakeontario_outflow_changes = lakeontario_outflow_changes.reindex(hourly_index)
lakeontario_outflow_changes = lakeontario_outflow_changes.ffill()

In [167]:
os.makedirs("./data/processed/", exist_ok=True)
lakeontario_outflow_changes.to_csv("./data/processed/LakeOntarioOutflowChanges_2024_HOURLY.csv")

Since the data do not have the same timestamps, it is difficult to use them for the purpose of computing values. Therefore we will interpolate and extrapolate data so we can put them all on the same timestamps for future calculations

In [168]:
# Convert datetime index to numeric values representing minutes since start
outflow_minutes = (lakeontario_outflow_changes.index - lakeontario_outflow_changes.index[0]).total_seconds() / 60
cornwall_minutes = (cornwall_water_lvl.index - cornwall_water_lvl.index[0]).total_seconds() / 60
longsault_minutes = (longsault_water_lvl.index - longsault_water_lvl.index[0]).total_seconds() / 60

# Create cubic interpolating functions
outflow_interpolation = interp1d(outflow_minutes, lakeontario_outflow_changes[OUTFLOW_COL], kind='cubic', fill_value="extrapolate")
longsault_water_lvl_interpolation = interp1d(longsault_minutes, longsault_water_lvl[WATER_LEVEL_COL], kind='cubic', fill_value="extrapolate")
cornwall_water_lvl_interpolation = interp1d(cornwall_minutes, cornwall_water_lvl[WATER_LEVEL_COL], kind='cubic', fill_value="extrapolate")

# Query at new timestamps
start = pd.Timestamp("2024-01-01 00:00:00")
end = pd.Timestamp("2025-01-01 00:00:00")
new_times = pd.date_range(start=start, end=end, freq="h")
new_minutes = (new_times - new_times[0]).total_seconds() / 60

# Get interpolated values
outflow_values = outflow_interpolation(new_minutes)
longsault_water_lvl_values = longsault_water_lvl_interpolation(new_minutes)
cornwall_water_lvl_values = cornwall_water_lvl_interpolation(new_minutes)

# Create new DataFrame
df_new = pd.DataFrame({
    "Qdot": outflow_values,
    "upstream": longsault_water_lvl_values,
    "downstream": cornwall_water_lvl_values
}, index=new_times)

Calculate the power for all these timestamps.

We first calculate the values of each parameters:
1. The `h` is calculated by substracting the upstream by the downstream
2. `x` is calculated to be open (`=1`) if the head `h` is positive and closed if negative (`x=0`). This is calculated for each timestamps ensuring a positive power value.
3. `ro` is a constant of 998 kg/m^3
4. `Qdot` is the outflow from the Lake Ontario. However the outflow value comes from the interpolated data
5. `g` is a constant of 9.81 m/s^2

With this the Energy `E` and Power `P` can be calculated:
1. `P = x * ro * Qdot * g * h`
2. `E = P * t`

The parameters are kept as a column in the dataframe including the Power in Watts and Energy in Watt seconds.

In [169]:
# calculate the head difference between upstream and downstream
h = (df_new["upstream"] - df_new["downstream"]).clip(upper=25, lower=0) # meters
# calculate the sluice gate
df_new["x"] = np.where(h <= 0, 0, 1)
df_new["h"] = h
# set values for power calculation
x = df_new["x"]
ro = 998 # kg/m^3
Qdot = df_new["Qdot"] # m^3/s
g = 9.81 # m/s^2

df_new["P"] = x * ro * Qdot * g * h
df_new["E"] = df_new["P"] * 3600  # Joules
df_new.loc[df_new.index[0], "E"] = 0 # Set the first value to 0

Create a new column that computes the revenue of each period

In [170]:
df_new["E (kWh)"] = df_new["E"] / 3600000  # kWh

winter_months = [11, 12, 1, 2, 3, 4] # November to April
summer_months = [5, 6, 7, 8, 9, 10] # May to October

off_peak_rate = 0.098  # $/kWh
mid_peak_rate = 0.157  # $/kWh
on_peak_rate = 0.203  # $/kWh

off_peak_hours = list(range(19, 24)) + list(range(0, 7))  # 7 PM to 7 AM

winter_mid_peak_hours = list(range(11, 17)) # 11 AM to 5 PM
summer_mid_peak_hours = list(range(7, 11)) + list(range(17, 19)) # 7 AM to 11 AM and 5 PM to 7 PM


def TOU_rate(timestamp):
    month = timestamp.month
    hour = timestamp.hour
    weekday = timestamp.weekday()  # Monday=0, Sunday=6

    # Weekend or off-peak hours
    if weekday >= 5 or hour in off_peak_hours:  # Weekend
        return off_peak_rate  # Off-peak rate
    
    # mid peak hours
    if hour in winter_mid_peak_hours and month in winter_months:
        return mid_peak_rate
    if hour in summer_mid_peak_hours and month in summer_months:
        return mid_peak_rate

    # on peak hours if not in any of the above
    return on_peak_rate
df_new["Rate ($/kWh)"] = df_new.index.map(TOU_rate)
df_new["Revenue ($)"] = df_new["E (kWh)"] * df_new["Rate ($/kWh)"]

In [171]:
df_new["Revenue ($)"]
total = df_new["Revenue ($)"].sum()
total

np.float64(2072289728.6597252)

In [172]:
df_new.to_csv("./data/processed/hydro_data_2024_hourly.csv")